In [2]:
# =======================================
# PASO 1: Importar Librerías
# =======================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# =======================================
# PASO 2: Cargar y Explorar el Dataset
# =======================================
file_path = "FIFA.csv"  # Asegúrate de que el archivo está en la misma carpeta
df = pd.read_csv(file_path, encoding="latin1")

# Mostrar primeras filas
print("Primeras filas del dataset:")
display(df.head())

# Información del dataset
print("Información general:")
df.info()

# Dimensiones del dataset
print(f"El dataset tiene {df.shape[0]} filas y {df.shape[1]} columnas.")

# =======================================
# PASO 3: Limpieza de Datos
# =======================================

# 1. Convertir Weight y Height a Kg y cm
df["Weight"] = df["Weight"].str.replace("lbs", "").astype(float) * 0.453592  # Convertir a Kg
df["Height"] = df["Height"].str.replace("'", ".").astype(float) * 30.48  # Convertir a cm

# 2. Eliminar "+1", "+2" en posiciones CDM, RDM...
df.iloc[:, 50:] = df.iloc[:, 50:].replace("\+\d", "", regex=True)

# 3. Arreglar columnas W/F, SM, IR (suponiendo que son categorías numéricas)
df["W/F"] = df["W/F"].astype(str).str.extract("(\d)").astype(float)
df["SM"] = df["SM"].astype(str).str.extract("(\d)").astype(float)
df["IR"] = df["IR"].astype(str).str.extract("(\d)").astype(float)

# 4. Agrupar Posiciones en Portero, Defensa, Mediocampista y Delantero
def posiciones_jugadores(pos):
    if pos in ["GK"]:
        return "Portero"
    elif pos in ["CB", "LB", "RB", "LWB", "RWB"]:
        return "Defensa"
    elif pos in ["CDM", "CM", "CAM", "LM", "RM"]:
        return "Mediocampista"
    else:
        return "Delantero"

df["Posición_Normalizada"] = df["BP"].apply(posiciones_jugadores)

# 5. Convertir Value y Wage a números enteros
def convertir_valor(valor):
    if isinstance(valor, str):
        if "M" in valor:
            return float(valor.replace("M", "")) * 1e6
        elif "K" in valor:
            return float(valor.replace("K", "")) * 1e3
        else:
            return float(valor)
    return np.nan

df["Value"] = df["Value"].apply(convertir_valor)
df["Wage"] = df["Wage"].apply(convertir_valor)

# =======================================
# PASO 4: Tratamiento de Valores Nulos
# =======================================
print("Valores nulos por columna:")
print(df.isnull().sum())

# Eliminar jugadores con demasiados valores nulos
df.dropna(thresh=df.shape[1] * 0.7, axis=0, inplace=True)

# Rellenar valores nulos numéricos con la media
df.fillna(df.mean(numeric_only=True), inplace=True)

print("Valores nulos tratados.")

# =======================================
# PASO 5: Detección y Manejo de Valores Extremos
# =======================================

# Estadísticas descriptivas
print("\n📈 Estadísticas descriptivas:")
display(df.describe())

# Visualizar valores extremos en la altura
plt.figure(figsize=(10, 5))
sns.boxplot(x=df["Height"])
plt.title("Valores Extremos en la Altura")
plt.show()

# Filtrar alturas absurdas (<140 cm y >210 cm)
df = df[(df["Height"] >= 140) & (df["Height"] <= 210)]

print("Valores extremos filtrados.")

# =======================================
# PASO 6: Análisis Descriptivo
# =======================================

# Comparar métricas por posición
df_grouped = df.groupby("Posición_Normalizada").agg({"Value": ["sum", "mean", "count"], "Height": "mean"})
print("Comparación por posición:")
display(df_grouped)

# Visualización de la comparación
plt.figure(figsize=(12, 6))
sns.barplot(x=df_grouped.index, y=df_grouped["Value"]["mean"], palette="viridis")
plt.ylabel("Valor Promedio (€)")
plt.title("Valor Promedio de Jugadores por Posición")
plt.show()

# Relación entre OVA y otras métricas
plt.figure(figsize=(12, 6))
sns.heatmap(df[["OVA", "Attacking", "Defending", "Value"]].corr(), annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlación entre OVA y otras métricas")
plt.show()

print("Análisis completo.")
